### TODO:
- [ ] latency summary에 추가하기
- [ ] frame drop baseline performance 비교하는 figure 만들기; 기존 (using summary)
- [ ] frame drop baseline들 backlog 증가 그래프 그리는 figure 만들기; 기존 (using data)
- [ ] offloading baseline performance 비교하는 figure 만들기; fraction & accuracy & latency 비교; simulation에서 쓰던 형태 (using data & summary)
- [ ] offloading baseline들 backlog 증가 그래프 그리는 figure 만들기; 기존 (using data)
- [ ] multi agent 성능 비교 하는 figure 만들기
- [ ] multi agent에서 각 source node의 backlog 증가 그래프 그려서 비교하기

### Import libraries

In [4]:
import os, sys, time, re, random
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
import seaborn as sns
import csv, json, ast  

In [4]:
### User define libraries
import visualization as vis
import warnings

warnings.filterwarnings("ignore")

### Define methods

In [ ]:

def extract_value_from_latency(value_str):
    return float(value_str.split('ms')[0].strip())

def get_latency_exp(csv_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        df['latency'] = df['latency'].apply(extract_value_from_latency)
        return df
        
def get_backlog_exp(csv_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        return df[['sum', 'avg']]
    
# def calculate_mean_latency(df):
#     mean_value = df['latency'].mean()
#     return mean_value

def calculate_mean_latency(df, length):
    mean_value = df['latency'].mean()
    if length > len(df) + 5:    
        sum_value = df['latency'].sum() 
        sum_value += (length-len(df)-5)*300000
        mean_value = sum_value / length
        
    return mean_value


### Define setting

In [7]:
experiment_path = 'data/experiment'
frame_drop_summary_path = 'data/summary/frame_dropping.csv'
computation_offloading_summary_path = 'data/summary/computation_offloading.csv'
multi_agent_summary_path = 'data/summary/multi_agent.csv'

dataset_list = ['JN', 'JK', 'SD']
exp_type_list = ['frame_dropping', 'computation_offloading', 'multi_agent']
frame_drop_method_list = ['LRLO', 'Reducto', 'FrameHopper', 'CAO']
computation_offloading_method_list = ['LRLO','JDPCRA', 'TLDOC']

#### 0. get average latency value

In [ ]:
MS_TO_S = 1000

for dataset in dataset_list:
    for exp_type in exp_type_list:
        if exp_type == 'frame_dropping':
            for method in frame_drop_method_list:
                base_dir = f'{experiment_path}/{exp_type}/{method}_{dataset}'
                
                if method != 'CAO':
                    subfolder_paths = [os.path.join(base_dir, name) for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]
                    for subfolder_path in subfolder_paths:
                        path_df = pd.read_csv(f'{subfolder_path}/path/path.csv')
                        job_length = len(path_df)
                        latency_file_path = f'{subfolder_path}/latency/test job 1.csv'
                        df = get_latency_exp(latency_file_path)
                        mean_latency = calculate_mean_latency(df, job_length) / MS_TO_S
                        print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}\t ({latency_file_path})')
                
                else:
                    path_df = pd.read_csv(f'{base_dir}/path/path.csv')
                    job_length = len(path_df)
                    latency_file_path = f'{base_dir}/latency/test job 1.csv'
                    df = get_latency_exp(latency_file_path)
                    mean_latency = calculate_mean_latency(df, job_length) / MS_TO_S
                    print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}')
        
        elif exp_type == 'computation_offloading':
            for method in computation_offloading_method_list:
                base_dir = f'{experiment_path}/{exp_type}/{method}_{dataset}'
                
                if method == 'LRLO':
                    subfolder_paths = [os.path.join(base_dir, name) for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]
                    for subfolder_path in subfolder_paths:
                        path_df = pd.read_csv(f'{subfolder_path}/path/path.csv')
                        job_length = len(path_df)
                        latency_file_path = f'{subfolder_path}/latency/test job 1.csv'
                        df = get_latency_exp(latency_file_path)
                        mean_latency = calculate_mean_latency(df, job_length) / MS_TO_S
                        print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}\t ({latency_file_path})')
                else:
                    path_df = pd.read_csv(f'{base_dir}/path/path.csv')
                    job_length = len(path_df)
                    latency_file_path = f'{base_dir}/latency/test job 1.csv'
                    df = get_latency_exp(latency_file_path)
                    mean_latency = calculate_mean_latency(df, job_length) / MS_TO_S
                
                    print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}')
                    
        # else:
        #     base_dir = f'{experiment_path}/{exp_type}/LRLO(MA)_{dataset}'
        #     latency_file_path_1 = f'{base_dir}/latency/test job 1.csv'
        #     latency_file_path_2 = f'{base_dir}/latency/test job 2.csv'    
            
        #     df = get_latency_exp(latency_file_path_1)
        #     mean_latency = calculate_mean_latency(df) / MS_TO_S
        #     print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}\t ({latency_file_path_1})')
            
        #     df = get_latency_exp(latency_file_path_2)
        #     mean_latency = calculate_mean_latency(df) / MS_TO_S
        #     print(f'{dataset}\t{exp_type}\t{method}\tmean latency: {mean_latency}\t ({latency_file_path_2})')

### 1. [Frame drop] performance 비교 (코드 완성)

In [ ]:
drop_df = pd.read_csv(frame_drop_summary_path, skipinitialspace=True)
drop_df

In [ ]:
from matplotlib.ticker import FuncFormatter


axis_size = 13
label_size = 15

drop_df['method_with_target'] = drop_df.apply(
    lambda row: "LRLO(Ours)" if row['method'] == "LRLO" else (f"{row['method']}({row['target_f1']})" if not pd.isna(row['target_f1']) else row['method']),
    axis=1
)
datasets = drop_df['dataset'].unique()

for dataset in datasets:
    subset = drop_df[drop_df['dataset'] == dataset]
    
    melted_df = subset.melt(id_vars=['method_with_target'], value_vars=['f1_score', 'fraction'], 
                            var_name='Metric', value_name='Value')

    fig, ax = plt.subplots(figsize=(7.2, 6))
    
    sns.barplot(data=melted_df, x='method_with_target', y='Value', hue='Metric', palette="viridis", ax=ax,  width=0.6, edgecolor='0')
    ax2 = ax.twinx()
    sns.lineplot(data=subset, x='method_with_target', y='latency', color="red", marker="o", label='latency', ax=ax2)

    # 축과 레이블 설정
    ax.set_xlabel('')
    ax.set_ylabel('Value (%)', fontsize=label_size)
    ax2.set_ylabel('Average latency (s)', fontsize=label_size)
    
    ax.tick_params(axis='x', rotation=35, labelsize=axis_size)

    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y * 100:.0f}'))  # 예: 0.1 -> 10
    
    # y축 tick 폰트 크기 조정
    ax.tick_params(axis='y', labelsize=axis_size)  
    ax2.tick_params(axis='y', labelsize=axis_size) 

    # 범례 폰트 크기 조정
    ax.legend(loc='upper left', fontsize=axis_size)
    ax2.legend(loc='upper right', fontsize=axis_size)

    plt.tight_layout()
    plt.show()

### 2. [Frame Drop] Backlog Graph

**2.1 backlog df 불러오기**

In [ ]:
backlog_df_dict = {
    'JN': {
        'LRLO(Ours)': [],
        'Reducto(0.7)': None,
        'Reducto(0.9)': None,
        'FrameHopper(0.7)': None,
        'FrameHopper(0.9)': None,
        'CAO': None
    },
    'JK': {
        'LRLO': [],
        'Reducto(0.7)': None,
        'Reducto(0.9)': None,
        'FrameHopper(0.7)': None,
        'FrameHopper(0.9)': None,
        'CAO': None
    },
    'SD': {
        'LRLO': [],
        'Reducto(0.7)': None,
        'Reducto(0.9)': None,
        'FrameHopper(0.7)': None,
        'FrameHopper(0.9)': None,
        'CAO': None
    }
}

for dataset in datasets:
    for method in frame_drop_method_list:
        if method == 'CAO':
            backlog_path = f'{experiment_path}/frame_dropping/{method}_{dataset}'
        elif method == 'LRLO':
            s

In [ ]:
# 전체 실험결과가 다 뽑고싶은 당신을 위해
skip = False
if not skip:
    mean_latency_list = []
    latency_df = {'JK':{}, 'SD':{}, 'JN':{}}
    backlog_df = {'JK':{}, 'SD':{}, 'JN':{}}

    data_dir_list = ['data/csvs/experiment/results_baseline/', 'data/csvs/experiment/results_LRLO/']
    dataset = ['JK', 'SD', 'JN']
            
    for data_dir in data_dir_list:
        for data in dataset:
            folder_path = os.path.join(data_dir, data)
            for root, dirs, files in os.walk(folder_path):
                for directory in dirs:
                    if directory == 'latency':
                        name = re.split(r"[/\\]",root)[-1]
                        
                        # parts = name.split("_")   
                        # prefix = parts[0] + "_" + name.split("_")[3]
                        if name.split("_")[0] == 'LRLO':
                            prefix = name.split("_")[0] + "_" + name.split("_")[3]
                        else:
                            if name.split("_")[0] == 'frameHopper':
                                prefix = "FrameHopper" + "_" + name.split("_")[2]
                            elif name.split("_")[0] == 'reducto':
                                prefix = "Reducto" + "_" + name.split("_")[2]
                        csv_path = os.path.join(root, directory, 'test job 1.csv')
                        #if prefix in latency_df[data]:
                        #    continue
                        latency_df[data][prefix]= get_latency_exp(csv_path)
                        path_path = os.path.join(root, 'path', 'path.csv')
                        df = pd.read_csv(path_path)
                        job_length = len(df)
                        mean_latency, latency_df[data][prefix] = calculate_mean_latency_exp(latency_df[data][prefix], job_length)
                        mean_latency_list.append([data+"_"+prefix, str(mean_latency)])
                    elif directory == 'backlog':
                        name = re.split(r"[/\\]",root)[-1]
                        if name.split("_")[0] == 'LRLO':
                            prefix = name.split("_")[0] + "_" + name.split("_")[3]
                        else:
                            if name.split("_")[0] == 'frameHopper':
                                prefix = "FrameHopper" + "_" + name.split("_")[2]
                            elif name.split("_")[0] == 'reducto':
                                prefix = "Reducto" + "_" + name.split("_")[2]
                        csv_path = os.path.join(root, directory, 'total_backlog.csv')
                        #if prefix in backlog_df[data]:
                        #    print("in")
                        #    continue
                        backlog_df[data][prefix] = get_backlog_exp(csv_path)

                    #print(prefix)
        
    with open("data/csvs/experiment/mean_latency.csv", "w",  newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(mean_latency_list)

In [ ]:
# key_list = ['frameHopper_0.7', 'frameHopper_0.9', 'reducto_0.7', 'reducto_0.9', 'LRLO_1000']
#lrlo_list = ['LRLO_1000', 'LRLO_10000', 'LRLO_100000', 'LRLO_1000000', 'LRLO_10000000', 'LRLO_100000000']
#for lrlo in lrlo_list:
#    print(lrlo)
key_list = ['FrameHopper_0.7', 'Reducto_0.7', "LRLO_1000000"]
columns = ['avg', 'sum']
target_data = 'JK'
sns.set_palette("muted")
for data in dataset:
    if data != target_data:
        continue
    data_df = backlog_df[data]
    for column in columns:
        #print(column)
        flag = True
        for key, df in data_df.items():
            #print(data_df.keys())
            if key_list[-1] not in data_df.keys():
                flag = False
                break
            #print(key)
            if key not in key_list:
                continue
            # x_values = np.arange(0, len(df) * 0.1, 0.1)
            df.reindex(np.arange(0, len(df) * 0.1, 0.1))
            sampled_df = dm.get_sampled_df(df, 10)
            #x_values = np.arange(0, len(sampled_df) * 0.1, 0.1)
            #sns.lineplot(x=latency_masking_df.index, y=latency_masking_df[column], alpha=0.5)
            if key[0] == 'L':
                key = "LRLO"
            sns.lineplot(x=sampled_df.index, y=sampled_df[column], label=key, lw=1.5, palette="Muted")
            #sns.lineplot(x=latency_unmasking_df.index, y=latency_unmasking_df[column], alpha=0.5)
        if flag:
            print("dataset", data)
            plt.xlim(0, 2800)
            plt.ylim(0, 300000000)
            plt.xlabel('Time (s)', fontsize=16)  
            plt.ylabel('Backlog (B)', fontsize=16) 
            plt.tick_params(labelsize=12) 
            plt.legend(loc='upper left', fontsize=12) 
            plt.savefig(SAVE_ROOT+"exp_backlog_"+column+".pdf") 
            idx += 1
            plt.show()
